This is a place holder for my code release

In [1]:
# from parsing_by_maxseminfo import parser
import sys
from parsing_by_maxseminfo import parser

# necessary to use prepackaged data
sys.modules['parser'] = parser

In [ ]:

# import parsing_by_maxseminfo.utils.prep
import argparse
import yaml 
import os
from easydict import EasyDict as edict
import torch
from transformers import TrainingArguments 
import sys

# %%
import numpy as np
import random
import torch
import lightning

# lightning.seed_everything(0)
"""
Training mode can be selected from one of the below
rl: SemInfo mean-baseline training with CRF as explained in the main text
nll: LL training as explained in the main text
a2c: Stepwise SemInfo training with CRF as explained in Appendix A1
a2c_v0: Posterior V0 training with CRF
ta2c_rules: Stepwise SemInfo training with PCFG
ta2c: Posterior V0 training with PCFG
tavg: Posterior mean-baseline training with PCFG

"""


training_mode = "a"
assert training_mode in ["rl", "nll", "a2c", "ta2c", "ta2c_rules", "a2c_v0", "tavg"]



# %%
f_config = "/home/chris/projects/tnpcfg-release/config/pas-grammar/english-ew-reward-tbtok-idf/npcfg_nt60_t120_en.spacy-10k-merged-0pas-fast-6-3-rlstart0.yaml"
input_args = [
    f"-c={f_config}",
    "--max_length=40", 
    f"--set_training_mode={training_mode}",
    "--set_min_span_reward=-4", 
    "--batch_size=4",
    "--unset_ptb_mode",
    "--ckpt_dir=./checkpoints/",
    "--set_mode_reward=log_tfidf",
    "--set_include_unary",
    # "--use_pcfg_samples",
    "--langstr=english",
]


# %%
from parsing_by_maxseminfo.utils.myargparse import get_argsndevice

# import train
args, device = get_argsndevice(input_args)


from parsing_by_maxseminfo.parser.helper.pas_grammar_data_helper import (
    DataModuleForPASCtrlPCFGReward,
)

derivative = args.model.model_name.split("-")[1]
dst = DataModuleForPASCtrlPCFGReward(
        hparams=args,
        langstr=args.langstr,
        use_cache=True,
        max_size=10000,
        merge_pas_data=False,
        pas_subsample=args.preprocessing_pas_subsample_count,
    )


/home/chris/projects/tnpcfg-release/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Namespace(conf='/home/chris/projects/tnpcfg-release/config/pas-grammar/english-ew-reward-tbtok-idf/npcfg_nt60_t120_en.spacy-10k-merged-0pas-fast-6-3-rlstart0.yaml', use_tf32=False, rank=0, ngpu=1, max_pasdata_lendiff=-1, max_bandwidth=4, pas_subsample_count=-1, alignment_coefficient=-1, adversarial_coefficient=-1, batch_size=4, debug=False, flag_use_spacy_preprocessing=False, langstr='english', use_ppl_loss=False, span_repr_mode='bge-m3', remark='none', forbid_bn=False, forbid_merged_nll=False, use_normalized_term_mlp=False, use_onesided=False, dropout=-1, force_fp32=False, max_length=40, wandb_tags=None, val_check_interval=5000, ckpt_dir='./checkpoints/', flag_curriculum_learning=False, flag_use_separate_nll_path=False, unset_logppl=False, unset_nll_weighing=False, set_fast_model=False, set_mode_offending_spans=False, ckpt=None, unset_renormalizing_marginals=False, set_lr=-1, unset_bert_mode=False, set_pas_suppression=False, wandb_project='Spanoverlap-PCFG', set_hit_count_threshold=-1

In [3]:
args.ckpt_dir

'./checkpoints/'

In [4]:
word_vocab = dst.word_vocab
print("working on vocab of size", word_vocab.vocab_size)

from parsing_by_maxseminfo.parser.model.C_PCFG import CompoundPCFGBaseline, CompoundPCFGPairwise
from parsing_by_maxseminfo.parser.model.TN_PCFG import TNPCFGBaseline

basemodel = args.model.model_name.split("-")[0]
from parsing_by_maxseminfo.parser.lightning_wrapper.LitNPCFG import (
    LitXNPCFGFCReward,
)

print(f"launching {args.model.model_name.split('-')}")

if basemodel in ["SNPCFG", "TNPCFG", "NPCFG", "CPCFG", "SCPCFG", "SNPCFGA2C", "NPCFGA2C", "CPCFGA2C"]:
    # raise NotImplementedError("No plan for TNPCFG experiments so fat")
    derivative = args.model.model_name.split("-")[1]
    print(f"launching {basemodel} {derivative}")
    model = LitXNPCFGFCReward(
            basemodel,
            args.model,
            word_vocab.vocab_size,
            args.experimental,
            args.optimizer,
            args.langstr,
        )
    
    # model = TNPCFGFixedCost(args.model, word_vocab.vocab_size, span_repr_mode="em", langstr = 'german').to(device)
else:
    raise NotImplementedError(f"{args.model.model_name} is not allowed")

# # %%
# args.test.batch_size = 4
# args.train.batch_size =4


# import Stemmer

# print(args.langstr)

working on vocab of size 10020
launching ['NPCFGA2C', 'FixedCostReward']
launching NPCFGA2C FixedCostReward
Constructing LitNPCFGFixedCost with experimental config {'alignment_coefficient': 1.0, 'adversarial_coefficient': 0.0, 'pas_subsample_count': 0, 'renormalizing_marginals': False, 'weigh_nll_loss': True, 'suppress_pas_contrib': False, 'flag_curriculum_learning': False, 'mode': 'nll', 'hit_count_threshold': 2, 'activation_flood': 0.001, 'mode_offending_spans': True, 'spancomp_loss_weight': 4.0, 'rl_warmup_steps': 5000, 'rl_start_step': 0, 'rl_initial_coeff': 0.0, 'rl_target_coeff': 1.0, 'rl_len_norm': False, 'apply_mean_baseline': True, 'maxent_initial_coeff': -0.01, 'maxent_target_coeff': -0.01, 'mode_reward': 'log_tfidf', 'min_span_reward': -4.0, 'include_unary': True, 'supervised_mode': False, 'sample_mode': 'crf'}


In [5]:

# stemmer = Stemmer.Stemmer(args.langstr)
class IdentityStemmer:
    def stemWords(self, words):
        return words

    def stemWord(self, word):
        return word


import lightning.pytorch as L
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks.progress import TQDMProgressBar

# Setup early stopping
early_stop_callback = EarlyStopping(
    monitor="val/sentence_f1",  # Metric to monitor
    min_delta=0.002,  # Minimum change to qualify as an improvement
    patience=args.train.patience,  # Number of epochs with no improvement after which training will be stopped
    verbose=True,
    mode="max",  # Minimize the monitored metric (use 'max' for metrics like accuracy)
)

In [6]:
# %%

train_dl, _ = dst.train_dataloader(
        # "english",
        # args.langstr,
        "null",
        max_len=40,
        min_len=3,
        device=device,
        pas_subsample_count=args.experimental.pas_subsample_count,
        flag_curriculum_learning=(
            args.experimental.flag_curriculum_learning
            if hasattr(args.experimental, "flag_curriculum_learning")
            else False
        ),
        mode_offending_spans=(
            args.experimental.mode_offending_spans
            if hasattr(args.experimental, "mode_offending_spans")
            else False
        ),
        bert_mode=(
            args.model.bert_mode if hasattr(args.model, "bert_mode") else "disabled"
        ),
        add_sentence_level_span=(
            args.experimental.add_sentence_level_span
            if hasattr(args.experimental, "add_sentence_level_span")
            else False
        ),
        min_span_reward=args.experimental.min_span_reward,  # min span reward must be specified
        mode_reward=(
            args.experimental.mode_reward
            if hasattr(args.experimental, "mode_reward")
            else "none"
        ),
        supervised_mode=(
            args.experimental.supervised_mode
            if hasattr(args.experimental, "supervised_mode")
            else False
        ),
    )

val_dl, _ = dst.dev_full_dataloader(
    args.langstr,
    max_len=100000,
    min_len=2,
    device=device,
    flag_respect_ptb_boundary=True,
    min_span_reward=args.experimental.min_span_reward,
    mode_reward=(
        args.experimental.mode_reward
        if hasattr(args.experimental, "mode_reward")
        else "none"
    ),
    # flag_respect_ptb_boundary=args.experimental.flag_respect_ptb_boundary if hasattr(args.experimental, 'flag_respect_ptb_boundary') else False,
)

test_dl, _ = dst.test_dataloader(
    args.langstr,
    max_len=1000000,
    min_len=2,
    device=device,
    # flag_respect_ptb_boundary=args.experimental.flag_respect_ptb_boundary if hasattr(args.experimental, 'flag_respect_ptb_boundary') else False,
    flag_respect_ptb_boundary=False,
)

train loader: add_sentence_level_span: False
train loader: reward mode: log_tfidf
dev full loader: add_sentence_level_span: False
train loader: reward mode: log_tfidf
finished pruning dataset, current dataset length 1690
sampling: current dataset size:  1690
Train Iter: add_sentence_level_span False
Train Iter: mode_offending_spans True
sampling: current dataset size:  2412


In [7]:
next(iter(val_dl))

({'target_form_array': [['influential',
    'members',
    'of',
    'the',
    'house',
    'ways',
    'and',
    'means',
    'committee',
    'introduced',
    'legislation',
    'that',
    'would',
    'restrict',
    'how',
    'the',
    'new',
    'savings-and-loan',
    'bailout',
    'agency',
    'can',
    'raise',
    'capital',
    'creating',
    'another',
    'potential',
    'obstacle',
    'to',
    'the',
    'government',
    "'s",
    'sale',
    'of',
    'sick',
    'thrifts'],
   ['we',
    "'re",
    'saying',
    'the',
    'worst',
    'thing',
    'that',
    'anyone',
    'can',
    'do',
    'is',
    'to',
    'see',
    'the',
    'market',
    'go',
    'down',
    'and',
    'dump',
    'everything',
    'which',
    'just',
    'drives',
    'the',
    'prices',
    'down',
    'further',
    'says',
    'john',
    'lampe',
    'painewebber',
    "'s",
    'director',
    'of',
    'advertising'],
   ['at',
    'jefferies',
    "'",
    'trading',


In [8]:
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint

best_sf1_checkpoint_callback = ModelCheckpoint(
    save_top_k=4,
    monitor="val/sentence_f1",
    
    mode="max",
    dirpath=args.ckpt_dir,
    filename="ckpt-sf1_{val/sentence_f1:.2f}",
)
saveall_checkpoint_callback = ModelCheckpoint(
    save_top_k=-1,
    dirpath=args.ckpt_dir,
    filename="ckpt-step_{step}",
)



In [9]:
from parsing_by_maxseminfo.parser.lightning_wrapper.scheduler import WarmupScheduler

rl_coeff_scheduler = WarmupScheduler(
    warmup_steps=(
        args.experimental.rl_warmup_steps
        if hasattr(args.experimental, "rl_warmup_steps")
        else 10000
    ),
    coeff_name="rl_coeff",
    initial_coeff=(
        args.experimental.rl_initial_coeff
        if hasattr(args.experimental, "rl_initial_coeff")
        else 0.0
    ),
    start_step=(
        args.experimental.rl_start_step
        if hasattr(args.experimental, "rl_start_step")
        else 20000
    ),
    target_coeff=(
        args.experimental.rl_target_coeff
        if hasattr(args.experimental, "rl_target_coeff")
        else 0.3
    ),
)

maxent_scheduler = WarmupScheduler(
    warmup_steps=(
        args.experimental.maxent_warmup_steps
        if hasattr(args.experimental, "maxent_warmup_steps")
        else 1
    ),
    coeff_name="maxent_coeff",
    initial_coeff=(
        args.experimental.maxent_initial_coeff
        if hasattr(args.experimental, "maxent_initial_coeff")
        else 0.5
    ),
    start_step=(
        args.experimental.maxent_start_step
        if hasattr(args.experimental, "maxent_start_step")
        else 0.0
    ),
    target_coeff=(
        args.experimental.maxent_target_coeff
        if hasattr(args.experimental, "maxent_target_coeff")
        else 0.5
    ),
)

In [10]:
max_steps = 100
min_steps = 100
val_check_interval = 100

In [11]:
assert not args.debug, "debug mode is not allowed in this version"
trainer = L.Trainer(
    # limit_train_batches=100,
    # # max_epochs=20,
    # min_epochs = 5,
    max_steps=max_steps,
    min_steps=min_steps,
    min_epochs=0,
    val_check_interval=val_check_interval,
    check_val_every_n_epoch=None,
    gradient_clip_val=args.train.clip,
    gradient_clip_algorithm="norm",
    callbacks=[
        early_stop_callback,
        TQDMProgressBar(refresh_rate=10),
        best_sf1_checkpoint_callback if not args.analysis_mode and not args.corr_mode else saveall_checkpoint_callback,
        rl_coeff_scheduler,
        maxent_scheduler,
    ],
    logger=[],  # if not args.debug else None,
    # devices=[args.rank],
    inference_mode=False,
    log_every_n_steps=10,
    accelerator="gpu",
    devices=1,          # Number of GPUs to use
    # strategy="ddp"      # Use Distributed Data Parallel
    
)
# wandb_logger.watch(model, log_graph=False, log_freq=100)
trainer.fit(
    model,
    train_dataloaders=train_dl,
    val_dataloaders=val_dl,
    ckpt_path=args.continue_from,
)



GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/chris/projects/tnpcfg-release/venv/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /data/chris/projects/tnpcfg-release/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name  | Type                         | Params | Mode 
---------------------------------------------------------------
0 | model | NeuralPCFGFixedCostRewardA2C | 24.5 M | train
---------------------------------------------------------------
24.5 M    Trainable params
0         Non-trainable params
24.5 M    Total params
98.046    Total estimated model params size (MB)
32        Modules in train mode
0         Modules in eval mode


Using Adam optimizer
Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  5.61it/s]total samples 8.0
dst_size 8
finished pruning dataset                                                   
Constructing shuffled batches with 50 epoches


100%|██████████| 1/1 [00:02<00:00,  2.95s/it]


Train Iter: add_sentence_level_span False
Train Iter: mode_offending_spans True
Epoch 0: |          | 0/? [00:00<?, ?it/s] 

/data/chris/projects/tnpcfg-release/parsing_by_maxseminfo/parser/model/N_PCFG.py:759: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1823.)
  stat_rw = rw.std(-1).mean(-1) #if not isinstance(rw, int) else 0


Epoch 0: |          | 100/? [00:24<00:00,  4.10it/s]

: 

In [12]:
if args.allow_ptb_eval:
    model.set_test_mode("ptb")
    print(
        "Training ends. The best model on PTB mode: \n",
        trainer.test(model, dataloaders=test_dl),
        file=sys.stderr,
    )
model.set_test_mode(mode="spacy")
print(
    "Training ends. The best model on Spacy mode: \n",
    trainer.test(model, dataloaders=test_dl),
    file=sys.stderr,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing DataLoader 0: 100%|██████████| 1223/1223 [01:31<00:00, 13.41it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        test/avg_ll        │    -144.63084411621094    │
│       test/avg_ppl        │     843.4830322265625     │
│      test/corpus_f1       │    0.22327956557273865    │
│     test/sentence_f1      │    0.2539501190185547     │
└───────────────────────────┴───────────────────────────┘

Training ends. The best model on Spacy mode: 
 [{'test/corpus_f1': 0.22327956557273865, 'test/sentence_f1': 0.2539501190185547, 'test/avg_ll': -144.63084411621094, 'test/avg_ppl': 843.4830322265625}]
